## Count how many comparisons developers have made

In [ ]:
import database

THRESHOLD = 1
PROJECT='torvalds-linux'

num_participants = 0
num_answers = 0
for c in database.stats_compared(PROJECT):
    if c[1] >= THRESHOLD:
        num_participants += 1
        num_answers += c[1]
        print(c[0], c[1])
print('# participants: %d\t# answers: %d' % (num_participants, num_answers))

In [ ]:
import database

PROJECT='torvalds-linux'
LINK='https://github.com/torvalds/linux/commit/'

comparisons = [c for c in database.list_compared(PROJECT)]
comparisons = sorted(comparisons, key=lambda x: x['commit1'])
for c in comparisons:
    print(LINK + c['commit1'], LINK + c['commit2'], c['reason'], sep='\t')

In [ ]:
import database
import random

PROJECT='torvalds-linux'
LINK='https://github.com/torvalds/linux/commit/'

comparisons = [c for c in database.list_compared(PROJECT)]
comparisons = sorted(comparisons, key=lambda x: x['commit1'])
for c in comparisons:
    if random.random() < 0.5:
        print(LINK + c['commit1'], LINK + c['commit2'], 0, sep='\t')
    else:
        print(LINK + c['commit2'], LINK + c['commit1'], 1, sep='\t')

In [ ]:
import common
import database

PROJECT='torvalds-linux'

commit2desc = dict()
commit2title = dict()


def add_to_dict(commit, title, desc):
    if commit not in commit2desc:
        commit2desc[commit] = [desc]
    elif desc not in commit2desc[commit]:
        commit2desc[commit].append(desc)
    
    if commit not in commit2title:
        commit2title[commit] = title
    else:
        assert commit2title[commit] == title


for c in database.list_compared(PROJECT):
    d1, d2 = common.parse_descriptions(c['reason'])
    add_to_dict(c['commit1'], c['title1'], '(+) ' + d1)
    add_to_dict(c['commit2'], c['title2'], '(-) ' + d2)
    
for i in commit2desc.items():
    print(i[0], commit2title[i[0]], i[1], sep='\t')

In [ ]:
import database

COMMIT = 'b1fa4dc4ffef843bda17f26bbf54fb5d8bc23d2b'


for i, a in enumerate(database.get_related_answers(COMMIT)):
    print('[%d] %s (%s)\t%s\t%s (%s)' %
          (i, a['commit1'], a['title1'], a['reason'], a['commit2'], a['title2']))